# Before you start:
- Read the README.md file
- Comment as much as you can and use the resources (README.md file)
- Happy learning!

In [40]:
# import numpy and pandas

import numpy as np
import pandas as pd
from scipy.stats import ttest_1samp
from scipy.stats import t, sem

# Challenge 1 - Exploring the Data

In this challenge, we will examine all salaries of employees of the City of Chicago. We will start by loading the dataset and examining its contents. Please, load the data using Ironhack's database (db: employees, table: employees_advanced).

In [2]:
# Your code here:
salaries = pd.read_csv('hypothesis-testing.csv')

Examine the `salaries` dataset using the `head` function below.

In [3]:
# Your code here:

salaries.head()

,Name,Job Titles,Department,Full or Part-Time,Salary or Hourly,Typical Hours,Annual Salary,Hourly Rate
0,"AARON, JEFFERY M",SERGEANT,POLICE,F,Salary,NaN,101442.0,NaN
1,"AARON, KARINA",POLICE OFFICER (ASSIGNED AS DETECTIVE),POLICE,F,Salary,NaN,94122.0,NaN
2,"AARON, KIMBERLEI R",CHIEF CONTRACT EXPEDITER,GENERAL SERVICES,F,Salary,NaN,101592.0,NaN
3,"ABAD JR, VICENTE M",CIVIL ENGINEER IV,WATER MGMNT,F,Salary,NaN,110064.0,NaN
4,"ABASCAL, REECE E",TRAFFIC CONTROL AIDE-HOURLY,OEMC,P,Hourly,20.0,NaN,19.86


We see from looking at the `head` function that there is quite a bit of missing data. Let's examine how much missing data is in each column. Produce this output in the cell below

In [4]:
# Your code here:

salaries.isnull().sum()

Name                     0
Job Titles               0
Department               0
Full or Part-Time        0
Salary or Hourly         0
Typical Hours        25161
Annual Salary         8022
Hourly Rate          25161
dtype: int64

Let's also look at the count of hourly vs. salaried employees. Write the code in the cell below

In [5]:
# Your code here:

salaried = salaries[salaries['Salary or Hourly'] == 'Salary']['Salary or Hourly'].shape[0]

hourly = salaries[salaries['Salary or Hourly'] == 'Hourly']['Salary or Hourly'].shape[0]


print(f'The number of salaried employees is {salaried} and hourly employees {hourly}')

The number of salaried employees is 25161 and hourly employees 8022


What this information indicates is that the table contains information about two types of employees - salaried and hourly. Some columns apply only to one type of employee while other columns only apply to another kind. This is why there are so many missing values. Therefore, we will not do anything to handle the missing values.

There are different departments in the city. List all departments and the count of employees in each department.

In [6]:
# Your code here:

salaries.groupby(['Department']).agg({'Name': 'count'}).reset_index()

,Department,Name
0,ADMIN HEARNG,39
1,ANIMAL CONTRL,81
2,AVIATION,1629
3,BOARD OF ELECTION,107
4,BOARD OF ETHICS,8
5,BUDGET & MGMT,46
6,BUILDINGS,269
7,BUSINESS AFFAIRS,171
8,CITY CLERK,84
9,CITY COUNCIL,411


# Challenge 2 - Hypothesis Tests

In this section of the lab, we will test whether the hourly wage of all hourly workers is significantly different from $30/hr. Import the correct one sample test function from scipy and perform the hypothesis test for a 95% two sided confidence interval.

In [8]:
# Your code here:

hourly = salaries[salaries['Salary or Hourly'] == 'Hourly']['Hourly Rate'].to_numpy()

test_results = ttest_1samp(hourly, 30)

display(test_results.statistic)
display(test_results.pvalue)

print('The probability that the hourly wage is different from $30/h is higher than 99%')

20.6198057854942

4.3230240486229894e-92

The probability that the hourly wage is different from $30/h is higher than 99%


We are also curious about salaries in the police force. The chief of police in Chicago claimed in a press briefing that salaries this year are higher than last year's mean of $86000/year a year for all salaried employees. Test this one sided hypothesis using a 95% confidence interval.

Hint: A one tailed test has a p-value that is half of the two tailed p-value. If our hypothesis is greater than, then to reject, the test statistic must also be positive.

In [19]:
# Your code here:

year_salary = salaries[salaries['Salary or Hourly'] == 'Salary']['Annual Salary'].to_numpy()

test_results = ttest_1samp(year_salary, 86000)

if (test_results.pvalue / 2) < 0.05:
    print('The salaries are higher than last year')
else:
    print('The salaries are smaller than last year')

The salaries are higher than last year


Using the `crosstab` function, find the department that has the most hourly workers. 

In [23]:
# Your code here:

pd.crosstab(salaries['Department'], salaries['Typical Hours'])

Typical Hours,10.0,20.0,35.0,40.0
Department,,,,
ANIMAL CONTRL,1,18,0,0
AVIATION,0,18,15,1049
BUDGET & MGMT,0,0,2,0
BUSINESS AFFAIRS,0,7,0,0
CITY COUNCIL,7,36,18,3
COMMUNITY DEVELOPMENT,0,3,1,0
CULTURAL AFFAIRS,0,0,7,0
FAMILY & SUPPORT,0,286,0,1
FINANCE,0,2,0,42


The workers from the department with the most hourly workers have complained that their hourly wage is less than $35/hour. Using a one sample t-test, test this one-sided hypothesis at the 95% confidence level.

In [35]:
# Your code here:

hourly = salaries[(salaries['Salary or Hourly'] == 'Hourly') & (salaries['Department'] == 'STREETS & SAN')]['Hourly Rate'].to_numpy()

test_results = ttest_1samp(year_salary, 86000)

if (test_results.pvalue / 2) < 0.05:
    print('The salaries are lower than $35/h')
else:
    print('It is unknown if the salary is lower than $35/h')

The salaries are lower than $35/h


# Challenge 3: To practice - Constructing Confidence Intervals

While testing our hypothesis is a great way to gather empirical evidence for accepting or rejecting the hypothesis, another way to gather evidence is by creating a confidence interval. A confidence interval gives us information about the true mean of the population. So for a 95% confidence interval, we are 95% sure that the mean of the population is within the confidence interval. 
).

To read more about confidence intervals, click [here](https://en.wikipedia.org/wiki/Confidence_interval).


In the cell below, we will construct a 95% confidence interval for the mean hourly wage of all hourly workers. 

The confidence interval is computed in SciPy using the `t.interval` function. You can read more about this function [here](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.stats.t.html).

To compute the confidence interval of the hourly wage, use the 0.95 for the confidence level, number of rows - 1 for degrees of freedom, the mean of the sample for the location parameter and the standard error for the scale. The standard error can be computed using [this](https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.sem.html) function in SciPy.

In [48]:
salaries[salaries['Salary or Hourly'] == 'Hourly']['Hourly Rate'].to_numpy()

array([19.86, 46.1 , 35.6 , ..., 35.6 , 46.35, 48.85])

In [50]:
# Your code here:
df = salaries[salaries['Salary or Hourly'] == 'Hourly'].shape[0] - 1
mean = salaries[salaries['Salary or Hourly'] == 'Hourly']['Hourly Rate'].mean()
standar_error = sem(salaries[salaries['Salary or Hourly'] == 'Hourly']['Hourly Rate'].to_numpy())

hourly_wage = t.interval(0.95, df, mean, standar_error)
hourly_wage

(32.52345834488425, 33.05365708767623)

Now construct the 95% confidence interval for all salaried employeed in the police in the cell below.

In [52]:
# Your code here:

df =  salaries[(salaries['Salary or Hourly'] == 'Salary') & (salaries['Department'] == 'POLICE')]['Annual Salary'].shape[0] - 1
mean = salaries[(salaries['Salary or Hourly'] == 'Salary') & (salaries['Department'] == 'POLICE')]['Annual Salary'].mean()
standar_error = sem(salaries[(salaries['Salary or Hourly'] == 'Salary') & (salaries['Department'] == 'POLICE')]['Annual Salary'].to_numpy())

salary_wage = t.interval(0.95, df, mean, standar_error)
salary_wage

(86177.05631531784, 86795.77269094894)

# Bonus Challenge - Hypothesis Tests of Proportions

Another type of one sample test is a hypothesis test of proportions. In this test, we examine whether the proportion of a group in our sample is significantly different than a fraction. 

You can read more about one sample proportion tests [here](http://sphweb.bumc.bu.edu/otlt/MPH-Modules/BS/SAS/SAS6-CategoricalData/SAS6-CategoricalData2.html).

In the cell below, use the `proportions_ztest` function from `statsmodels` to perform a hypothesis test that will determine whether the number of hourly workers in the City of Chicago is significantly different from 25% at the 95% confidence level.

In [ ]:
# Your code here:

